In [ ]:
from ast import If
import json
import csv
import os
import glob
import numpy as np
import playerData

jsonFiles = []
csvFiles = []
playersArray = []
numNiveles = 4
nivel = -1
tiempoInicioNivel = 0
muertesPorNivel = np.zeros(numNiveles) #esto mide la suma de todas las muertes hechas por todos los jugadores?¿
saltosPorNivel = np.zeros(numNiveles)  #esto mide la suma de todos los saltos hechos por todos los jugadores?¿
numArchivos = 0
zonasEspeciales = None

def initZonasEspeciales():
    global zonasEspeciales
    zonasEspeciales = [[] for i in range(numNiveles)]
    zonasEspeciales[0].append([[23, 3], [26, 0]])

def getCollision(x, y, rect):
                #rect->arribaizq->x
        return (x > rect[0][0] and
                #rect->abajodcha->x
                x < rect[1][0] and
                #rect->arribaizq->y
                y < rect[0][1] and
                #rect->abajodcha->y
                y > rect[1][1])

def ProcessJson(f):
    Lines = f.readlines()
    count = 0
    # For each line process one event
    events = []
    for line in Lines:
        count += 1
        events.append(json.loads(line))
    return events

def ProcessCSV(f):

    players = []

    ncol = len(next(f))
    start = 3

    for row in f:
        answers = []
        name = str(row[start - 1])
        name = "".join([ch for ch in name if ch.isdigit()])   # getONLY the numbers in the name
        
        for x in range(start, ncol):
            answers.append(row[x])
        player = name, answers
        players.append(player)
    
    return players



# def Clean(f):
#     session_start = []
#     for i in range(0, f.readlines, 1):
#         line = f.readline[i]   

def ReadFiles():
    #List of files in folder
    for filename in glob.glob('**/*.json'):
        with open(filename, mode="r", encoding="utf-8") as f: # open files in readonly mode
            fName = f.name
            fName = "".join([ch for ch in fName if ch.isdigit()])   # getONLY the numbers in the name
            file = str(fName), ProcessJson(f)
            jsonFiles.append(file)
            player = playerData.Player(numNiveles, zonasEspeciales)
            playersArray.append(player)
            global numArchivos
            numArchivos += 1
            
    for filename in glob.glob('*.csv'):
        with open(filename, mode="r", encoding="utf-8") as f:
            csvreader = csv.reader(f)
            file = str(f.name), ProcessCSV(csvreader)
            csvFiles.append(file)
            


In [ ]:
def SessionStart(e,player):
    ID = e["sesion"]
    player.setID(ID)
def SessionEnd(e,player):
    player.end()
def LevelStart(e,player):
    global nivel
    nivel = e["nivel"] -3 #Los niveles estan codificados como nivel 3, 4, 5 y 6, asi que les resto esto para que se guarden como 0,1,2, y 3.
    player.resetPath(nivel)

    #si no ha llegado al final del nivel, pero si lo ha empezado antes (p.ej: si se buggea y reinicia), borra los datos de ese nivel
    if (not player.hasLevelBeenCompleted(nivel)):
        player.resetLevelInfo(nivel)
        global tiempoInicioNivel
        tiempoInicioNivel = e["tiempo"]

def LevelEnd(e, player):
    player.lvlEnd(e["tiempo"] - tiempoInicioNivel)

def Pause(e, player):
    player.pause(e["tiempo"])

def Unpause(e, player):
    player.pause(e["tiempo"])

def Death(e, player):
    muertesPorNivel[nivel] += 1
    player.death(nivel)

def Collision(e, player):
    player.collision(e["tag"], e["id"])


def Jump(e, player):
    saltosPorNivel[nivel] += 1
    player.jump(nivel)
    for zona in range(len(zonasEspeciales[nivel])):
        if (getCollision(e["x"], e["y"], zonasEspeciales[nivel][zona])):
            player.jumpEspecial(nivel, zona)


def PlayerPosition(e, player):
    pos = e["x"], e["y"]
    player.addToPath(pos, nivel)

In [ ]:
def FormularioEnd(p):
        player = playersArray[p]
        player.formularioInforme()

def Age(a, player):
        player.setAge(a)

def GeneralExperience(a, player):
        player.setGeneralExperience(a)

def PlatformExperience(a, player):
        player.setPlatformExperience(a)

def Walljump(a, player):
        player.setWalljump(a)

def Gravity(a, player):
        player.setGravity(a)
        
def LevelDificulty(level, a, player):
        player.setLevelDificulty(level, a)
        
def PlatformExperience(a, player):
        player.setPlatformExperience(a)

def Walljump(a, player):
        player.setWalljump(a)

def Gravity(a, player):
        player.setGravity(a)
        
def LevelDificulty(level, a, player):
        player.setLevelDificulty(level, a)

In [ ]:
def ProcessEvent(e,actualPlayer):
     nombre = e["nombre"]
     player = playersArray[actualPlayer] #algunos eventos usaran al jugador y otros no

     if nombre == "LEVEL_START":
          LevelStart(e,player)

     elif nombre == "SESSION_START":
          SessionStart(e,player)
          
     elif nombre == "SESSION_END": #descomentar para leer informes individuales
          SessionEnd(e,player)

     #no tiene en cuenta los eventos de debajo si ya ha completado el nivel antes
     if (player.hasLevelBeenCompleted(nivel)): 
          return

     if nombre == "LEVEL_END":
          LevelEnd(e, player)


     elif nombre == "PAUSE":
          Pause(e, player)

     elif nombre == "UNPAUSE":
          Unpause(e, player)

     elif nombre == "DEATH":
          Death(e,player)

     elif nombre == "COLLISION":
          Collision(e, player)

     elif nombre == "JUMP":
          Jump(e,player)

     elif nombre == "PLAYER_POSITION":
          PlayerPosition(e,player)


     #print(nivel,":",nombre)




In [ ]:
def ProcessAnswer(a, actualAnswer, actualPlayer):
    player = playersArray[actualPlayer]

    if(actualAnswer == 0):
        Age(a, player)

    elif(actualAnswer == 1):
        GeneralExperience(a, player)
        
    elif(actualAnswer == 2):
        PlatformExperience(a, player)
        
    elif(actualAnswer == 3):
        Walljump(a, player)
    
    elif(actualAnswer == 4):
        Gravity(a, player)
    
    elif(actualAnswer >= 5):
        LevelDificulty(actualAnswer - 5, a, player)
        

In [ ]:

def ProcessFiles():
    initZonasEspeciales()
    ReadFiles()
    actualPlayer = -1 #iterador para añadir datos de jugador
    for f in jsonFiles:
        actualPlayer+=1
        print("Processing player", actualPlayer)
        for e in f[1]:
            ProcessEvent(e,actualPlayer)
    
    for f in csvFiles:
        for p in f[1]:
            index = 0
            found = False
            actualPlayer = 0

            for n in playersArray:
                if str(n.ID) == p[0]:
                    actualPlayer = index
                    found = True
                    break
                else: index += 1

            if found == True:
                print("Processing formulary", actualPlayer)
                actualAnswer= 0
                for a in p[1]:
                    ProcessAnswer(a, actualAnswer, actualPlayer)
                    actualAnswer += 1
                FormularioEnd(actualPlayer)
            else: print("Wrong ID: ", p[0], "\n---------------------------------------------")
                
            
            
    print("------------------------------------------")
    print("Media de saltos por jugador en cada nivel:",saltosPorNivel/np.size(playersArray))
    print("Media de muertes por jugador en cada nivel:",muertesPorNivel/np.size(playersArray))

ProcessFiles()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.misc as sci
import imageio
from PIL import Image

im = imageio.imread("guille.jpg")
plt.imshow(im)
#Create the heatmap
kde = sns.kdeplot(
        map(lambda x: x[0],  playersArray[0].paths[3]),
        map(lambda x: x[1],  playersArray[0].paths[3]),
        shade = True,
        shade_lowest=False,
        alpha=.5,
        n_levels=10,
        cmap = 'magma'
)

plt.show()